In [1]:
# !pip3 install transformers>=4.32.0 optimum>=1.12.0
# !pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7


In [2]:
!pip install accelerate optimum -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
# !pip install transformers>=4.33.0 optimum>=1.12.0

In [4]:
# !pip install optimum -q

In [5]:
pip install auto-gptq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.9 MB/s eta 0:00:00


In [6]:
import torch
print(torch.__version__)

2.2.1+cu121


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [9]:
import json

new_data = []
# Load JSON file
with open('/content/drive/MyDrive/CSCI 544/Project/dataset/strategy_qa.json', 'r') as file:
    data = json.load(file)

# for i,j in enumerate(data['data']):
#   if i<300:
#     continue
#   new_data.append(j)
# Initialize an empty list to store results



In [10]:
new_data = data['data'][1000:]

In [ ]:
count = 0
results = []
for item in new_data:
    # Get prompt details
    prompt = item
    count= count+1
    print(count)
    prompt_template=f'''[INST]<<SYS>>Lets think step by step. {item['question']}[/INST]'''
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.1,
        top_p=0.95,
        top_k=1,
        repetition_penalty=1.1
    )
    response = pipe(prompt_template)[0]['generated_text']
    response = response.split('[/INST]')[1].strip()

    results.append({'prompt': item['question'], 'reasoning': response, 'answer': item['answer']})

    if count==300:
      break


In [ ]:
import json

In [ ]:
with open('/content/drive/MyDrive/CSCI 544/Project/Training_datasets/strategy_qa_1000_1600.json', 'a') as file:
     json.dump(results, file, indent=4)

In [ ]:
# # prompt = "Joan found 70 seashells on the beach . she gave Sam some of her seashells . She has 27 seashell . How many seashells did she give to Sam ?"
# prompt = '''{
#             "sample_index": 1,
#             "question": "There were 28 bales of hay in the barn . Tim stacked bales in the barn today . There are now 54 bales of hay in the barn . How many bales did he store in the barn ?",
#             "answer": "26"
#         }'''

# prompt_template=f'''[INST] <<SYS>>
# You are an online math tutor bot. Follow the given instructions: Given the below-structured query,
# your job is to understand the word problem mentioned as "question" and explain the solution in numbered
# steps to get the correct "answer". Begin your response with "let's think step by step" and append Answer: ["answer" from the
# below-structured query] at the end of your reasoning. Strictly limit the reasoning to 20-30 words. Do not use emojis.{prompt}[/INST]
# '''

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.1,
#     top_p=0.95,
#     top_k=5,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] <<SYS>>
You are an online math tutor bot. Follow the given instructions: Given the below-structured query, 
your job is to understand the word problem mentioned as "question" and explain the solution in numbered 
steps to get the correct "answer". Begin your response with "let's think step by step" and append Answer: ["answer" from the 
below-structured query] at the end of your reasoning. Strictly limit the reasoning to 20-30 words. Do not use emojis.{
            "sample_index": 1,
            "question": "There were 28 bales of hay in the barn . Tim stacked bales in the barn today . There are now 54 bales of hay in the barn . How many bales did he store in the barn ?",
            "answer": "26"
        }[/INST]
  Sure! Let's think step by step:

1. We know there were 28 bales of hay in the barn initially.
2. Tim stacked bales in the barn today, so we need to find out how many bales he stacked.
3. The total number of bales in the barn now is 54.
4. To find out h